# Agents(代理)
您可以将一个Runnable传递给代理程序。

In [145]:
from langchain.agents import AgentExecutor, XMLAgent, tool
from langchain.chat_models import ChatOpenAI

In [146]:
model = ChatOpenAI(
    openai_api_key="",
    openai_api_base=""
)

In [147]:
@tool
def search(query:str)->str:
    """Search things about current events."""
    return "32 degrees"

In [148]:
tool_list = [search]

In [149]:
# 获取代理的默认提示符
prompt = XMLAgent.get_default_prompt()

In [156]:
# 从中间步骤到传递给模型的字符串的逻辑
# 这与提示密切相关

def convert_intermediate_steps(intermediate_steps):
    log = ""
    # print("intermediate_steps:",intermediate_steps)
    for action,observation in intermediate_steps:
        log +=(
            f"<tool>{action.tool}</tool>"
            f"<tool_input>{action.tool_input}</tool_input>"
            f"<observation>{observation}</observation>"
        )
    return log


# 将工具转换为字符串的逻辑以便放入提示符
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

通常，从可运行的代码构建代理需要做一些事情：
1. 中间步骤的数据处理。这些数据需要以语言模型能够识别的方式表示。
2. 这应该与prompt的指令紧密耦合。
3. 模型（如有必要，包括停止标记）.
4. 输出解析器 - 应与提示指定的格式相一致。

In [151]:
agent = (
    {
        "question": lambda x: x["question"],
        "intermediate_steps": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgent.get_default_output_parser()
)

In [152]:
agent_executor = AgentExecutor(agent=agent,tools=tool_list, debug=True,verbose=True)

In [153]:
agent_executor.invoke({"question": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>weather in New York32 degrees<final_answer>The weather in New York is 32 degrees

> Finished chain.


{'question': 'whats the weather in New york?',
 'output': 'The weather in New York is 32 degrees'}